In [127]:
import requests, re
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time, random, pickle
import Levenshtein as lev
from urllib.request import urlopen
import urllib.parse, os
import numpy as np
from IPython.display import display

In [2]:
fig_path = '/Users/samuelblass/Desktop/Project/3. Business/Figures/'

In [3]:
def get_soup_driver(base_url): # Retrieves soup from website using Selenium
#     driver = webdriver.Chrome()
    driver.get(base_url)
    time.sleep(10 + random.uniform(-1,1))
    r = driver.page_source
    
#     driver.quit()
    return bs(r, 'html.parser')  

In [165]:
def get_zips(city, state, scraped_zips):
    city = urllib.parse.quote(city, safe='~')
    base_url = f'https://www.zip-codes.com/search.asp?fld-city={city}&fld-state={state}'
    soup = get_soup_driver(base_url)
    table = soup.find_all('table')
    df = pd.read_html(str(table))[1]
    city = urllib.parse.unquote(city)
    df['Levenshtein'] = df.City.apply(lambda x: lev.ratio(x, city.upper()))
    df['Zip Code'] = df['Zip Code'].apply(lambda x: str(x))
    df['Zip Code'] = df['Zip Code'].apply(lambda x: '0' + str(x) if len(str(x)) == 4 else x)
    
    scraped_zips[city+' '+state] = df.to_dict()
    pickling_on = open(fig_path + "scraped_zips.pickle","wb")
    pickle.dump(scraped_zips, pickling_on)
    pickling_on.close()
#     return df[df.Levenshtein > 0.9]
    return df[['City', 'State', 'Zip Code']][df.Levenshtein > 0.9]

In [164]:
driver = webdriver.Chrome()
df = get_zips('New York', 'NY', scraped_zips)
driver.quit()
df

,Zip Code,Area Code,City,County,State,Country,Levenshtein
0,10001,646/718/332/917/929/212/347,NEW YORK,NEW YORK,NY,US,1.0
1,10002,718,NEW YORK,NEW YORK,NY,US,1.0
2,10003,212/646/347/917/332/718/929,NEW YORK,NEW YORK,NY,US,1.0
3,10004,212/917/718/646/347,NEW YORK,NEW YORK,NY,US,1.0
4,10005,212/929/646/917/332/718/347,NEW YORK,NEW YORK,NY,US,1.0
5,10006,917/929,NEW YORK,NEW YORK,NY,US,1.0
6,10007,929/212/646/917/347/332/718,NEW YORK,NEW YORK,NY,US,1.0
7,10008,212/929/332/646/917,NEW YORK,NEW YORK,NY,US,1.0
8,10009,212/718/332/646/917/929,NEW YORK,NEW YORK,NY,US,1.0
9,10010,917/646/212/332/718/929,NEW YORK,NEW YORK,NY,US,1.0


In [171]:
def loop_zips(path_city_list, path_zip_list, start, scraped_zips): # put in path to zip code df in progress and start index
    city_df = pd.read_csv(path_city_list, index_col=0) # this is df master list of cities to be searched for zip
    zip_df = pd.read_csv(path_zip_list, index_col=0) # this is df containing zips that gets added onto each loop
    
    cities = list(city_df.City)[start:]
#     print(cities)
    states = list(city_df.State)[start:]
    indices = list(city_df.index)[start:]
    
    global driver
    driver = webdriver.Chrome()
    
    counter = 0
    
    for i, city in enumerate(cities):
        try:
            sub_df = get_zips(city, states[i], scraped_zips)
            zip_df = zip_df.append(sub_df, ignore_index=True)
            city_df.loc[indices[i], 'Done'] = 1
            city_df.to_csv(path_city_list)
            zip_df.to_csv(path_zip_list)
            counter += 1
#             if counter == 2: break
        except IndexError:
            driver.quit()
            return df
        except:
            continue
    driver.quit()
    return df
    #     print(start)
#     display(df) 
        
    

In [172]:
df = loop_zips(fig_path + 'cities.csv', fig_path + 'zips.csv', 15, scraped_zips)
df
# commuter_df_NYNJCTPA

,Zip Code,Area Code,City,County,State,Country,Levenshtein
0,10001,646/718/332/917/929/212/347,NEW YORK,NEW YORK,NY,US,1.0
1,10002,718,NEW YORK,NEW YORK,NY,US,1.0
2,10003,212/646/347/917/332/718/929,NEW YORK,NEW YORK,NY,US,1.0
3,10004,212/917/718/646/347,NEW YORK,NEW YORK,NY,US,1.0
4,10005,212/929/646/917/332/718/347,NEW YORK,NEW YORK,NY,US,1.0
5,10006,917/929,NEW YORK,NEW YORK,NY,US,1.0
6,10007,929/212/646/917/347/332/718,NEW YORK,NEW YORK,NY,US,1.0
7,10008,212/929/332/646/917,NEW YORK,NEW YORK,NY,US,1.0
8,10009,212/718/332/646/917/929,NEW YORK,NEW YORK,NY,US,1.0
9,10010,917/646/212/332/718/929,NEW YORK,NEW YORK,NY,US,1.0


In [185]:
def run_loop_zips(cities_path, zips_path, scraped_zips, counter=0):
    df = pd.read_csv(cities_path, index_col=0)
    df.reset_index(drop=True, inplace=True)
    start_index = list(df[df.Done.isna()].index)[0]
    try:
        loop_zips(cities_path, zips_path, start_index, scraped_zips)
    except:
        counter += 1
        if counter == 5:
            print(counter)
            raise Exception
        time.sleep(5)
        run_loop_zips(cities_path, zips_path, scraped_zips, counter)

In [189]:
run_loop_zips(fig_path + 'cities.csv', fig_path + 'zips.csv', scraped_zips)

In [181]:
df = pd.read_csv(fig_path + 'cities.csv', index_col=0)
df.reset_index(drop=True, inplace=True)
start_index = list(df[df.Done.isna()].index)[0]
display(df)
print(start_index)

,City,State,Done
0,New York,NY,1.0
1,Philadelphia,PA,1.0
2,Pittsburgh,PA,1.0
3,Buffalo,NY,1.0
4,Newark,NJ,1.0
...,...,...,...
1056,McConnellsburg,PA,NaN
1057,East Garden,NY,NaN
1058,Mifflintown,PA,NaN
1059,Wrightstown,NJ,NaN


15


In [74]:
pd.read_csv(fig_path + 'zips.csv', index_col=0)

,City,State,Zip Code
0,NaN,NaN,NaN
1,New York,NY,10001.0
2,New York,NY,10002.0
3,New York,NY,10003.0
4,New York,NY,10004.0
...,...,...,...
96,Philadelphia,PA,19142.0
97,Philadelphia,PA,19143.0
98,Philadelphia,PA,19144.0
99,Philadelphia,PA,19145.0


In [41]:
df.iloc[47:53]

,Rank,Area Name,St1,Size,St2,PlcCode,Population,WorkersWP,WorkersLP,Daytime,Daytime1,%Daytime1,WorkersLW,%WorkersLW,Zip Code_x,Zip Code_y,City,State
47,1,New York,NY,1,36,51000,"8,008,278","3,755,130","3,192,070","8,571,338","563,060",7.0,"2,922,206",91.5,NaN,10075.0,New York,NY
48,1,New York,NY,1,36,51000,"8,008,278","3,755,130","3,192,070","8,571,338","563,060",7.0,"2,922,206",91.5,NaN,10080.0,New York,NY
49,1,New York,NY,1,36,51000,"8,008,278","3,755,130","3,192,070","8,571,338","563,060",7.0,"2,922,206",91.5,NaN,10081.0,New York,NY
50,5,Philadelphia,PA,1,42,60000,"1,517,550","659,991","569,761","1,607,780","90,230",5.9,"429,608",75.4,NaN,NaN,NaN,NaN
51,52,Pittsburgh,PA,3,42,61000,"334,563","280,035","141,844","472,754","138,191",41.3,"98,005",69.1,NaN,NaN,NaN,NaN
52,58,Buffalo,NY,3,36,11000,"292,648","158,245","110,640","340,253","47,605",16.3,"65,989",59.6,NaN,NaN,NaN,NaN


In [12]:
driver = webdriver.Chrome()
nyc_zips = get_zips('New York', 'NY')
nyc_zips.to_pickle(fig_path + 'nyc_zips')
driver.quit()

In [13]:
nyc_zips = pd.read_pickle(fig_path + 'nyc_zips')

In [59]:
nyc_zips

,Zip Code,City,State
0,10001,New York,NY
1,10002,New York,NY
2,10003,New York,NY
3,10004,New York,NY
4,10005,New York,NY
5,10006,New York,NY
6,10007,New York,NY
7,10008,New York,NY
8,10009,New York,NY
9,10010,New York,NY


In [22]:
driver = webdriver.Chrome()
philly_zips = get_zips('Philadelphia', 'PA')
philly_zips.to_pickle(fig_path + 'philly_zips')
driver.quit()

In [42]:
philly_zips = pd.read_pickle(fig_path + 'philly_zips')

In [34]:
testNYC = testNYC.rename(columns={'Area Name':'City', 'St1': 'State'})

In [44]:
commuter_df_NYNJCTPA = commuter_df_NYNJCTPA.rename(columns={'Area Name':'City', 'St1': 'State'})
commuter_df_NYNJCTPA

,Rank,City,State,Size,St2,PlcCode,Population,WorkersWP,WorkersLP,Daytime,Daytime1,%Daytime1,WorkersLW,%WorkersLW
0,1,New York,NY,1,36,51000,"8,008,278","3,755,130","3,192,070","8,571,338","563,060",7.0,"2,922,206",91.5
4,5,Philadelphia,PA,1,42,60000,"1,517,550","659,991","569,761","1,607,780","90,230",5.9,"429,608",75.4
51,52,Pittsburgh,PA,3,42,61000,"334,563","280,035","141,844","472,754","138,191",41.3,"98,005",69.1
57,58,Buffalo,NY,3,36,11000,"292,648","158,245","110,640","340,253","47,605",16.3,"65,989",59.6
62,63,Newark,NJ,3,34,51000,"273,546","147,395","87,720","333,221","59,675",21.8,"36,319",41.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6439,6440,McConnellsburg borough,PA,9,42,46000,"1,073","2,875",478,"3,470","2,397",223.4,227,47.5
6444,6445,East Garden CDP,NY,9,36,22065,979,"23,591",485,"24,085","23,106",2360.2,38,7.8
6449,6450,Mifflintown borough,PA,9,42,49304,861,"3,630",396,"4,095","3,234",375.6,207,52.3
6453,6454,Wrightstown borough,NJ,9,34,82960,748,"6,698",349,"7,097","6,349",848.8,75,21.5


In [84]:
cities_df = commuter_df_NYNJCTPA[['City', 'State']]
cities_df.City = cities_df.City.apply(lambda x: x.replace('CDP', '').replace('(balance)', '').
                                      replace('borough', '').strip())
cities_df['Done'] = np.nan
cities_df.to_csv(fig_path + 'cities.csv')

/var/folders/b1/x16nkyxn74s8p16m4l_9dqh00000gn/T/ipykernel_13174/2897293536.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities_df['Done'] = np.nan


In [56]:
test = commuter_df_NYNJCTPA.merge(nyc_zips, on=['City', 'State'], how='left').merge(philly_zips, on=['City', 'State'], how='left')
test

,Rank,City,State,Size,St2,PlcCode,Population,WorkersWP,WorkersLP,Daytime,Daytime1,%Daytime1,WorkersLW,%WorkersLW,Zip Code_x,Zip Code_y
0,1,New York,NY,1,36,51000,"8,008,278","3,755,130","3,192,070","8,571,338","563,060",7.0,"2,922,206",91.5,10001.0,NaN
1,1,New York,NY,1,36,51000,"8,008,278","3,755,130","3,192,070","8,571,338","563,060",7.0,"2,922,206",91.5,10002.0,NaN
2,1,New York,NY,1,36,51000,"8,008,278","3,755,130","3,192,070","8,571,338","563,060",7.0,"2,922,206",91.5,10003.0,NaN
3,1,New York,NY,1,36,51000,"8,008,278","3,755,130","3,192,070","8,571,338","563,060",7.0,"2,922,206",91.5,10004.0,NaN
4,1,New York,NY,1,36,51000,"8,008,278","3,755,130","3,192,070","8,571,338","563,060",7.0,"2,922,206",91.5,10005.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,6440,McConnellsburg borough,PA,9,42,46000,"1,073","2,875",478,"3,470","2,397",223.4,227,47.5,NaN,NaN
1155,6445,East Garden CDP,NY,9,36,22065,979,"23,591",485,"24,085","23,106",2360.2,38,7.8,NaN,NaN
1156,6450,Mifflintown borough,PA,9,42,49304,861,"3,630",396,"4,095","3,234",375.6,207,52.3,NaN,NaN
1157,6454,Wrightstown borough,NJ,9,34,82960,748,"6,698",349,"7,097","6,349",848.8,75,21.5,NaN,NaN


In [58]:
test.iloc[:,-2].combine_first(test.iloc[:,-1]).iloc[:100]

0     10001.0
1     10002.0
2     10003.0
3     10004.0
4     10005.0
       ...   
95    19142.0
96    19143.0
97    19144.0
98    19145.0
99    19146.0
Name: Zip Code_x, Length: 100, dtype: float64

In [16]:
commuter_df_NYNJCTPA = pd.read_pickle(fig_path + 'commuter_df_NYNJCTPA')
commuter_df_NYNJCTPA.to_csv(fig_path + 'commuter_df_NYNJCTPA.csv')

In [17]:
commuter_df_NYNJCTPA = pd.read_csv(fig_path + 'commuter_df_NYNJCTPA.csv', index_col=0)
commuter_df_NYNJCTPA

,Rank,Area Name,St1,Size,St2,PlcCode,Population,WorkersWP,WorkersLP,Daytime,Daytime1,%Daytime1,WorkersLW,%WorkersLW
0,1,New York,NY,1,36,51000,"8,008,278","3,755,130","3,192,070","8,571,338","563,060",7.0,"2,922,206",91.5
4,5,Philadelphia,PA,1,42,60000,"1,517,550","659,991","569,761","1,607,780","90,230",5.9,"429,608",75.4
51,52,Pittsburgh,PA,3,42,61000,"334,563","280,035","141,844","472,754","138,191",41.3,"98,005",69.1
57,58,Buffalo,NY,3,36,11000,"292,648","158,245","110,640","340,253","47,605",16.3,"65,989",59.6
62,63,Newark,NJ,3,34,51000,"273,546","147,395","87,720","333,221","59,675",21.8,"36,319",41.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6439,6440,McConnellsburg borough,PA,9,42,46000,"1,073","2,875",478,"3,470","2,397",223.4,227,47.5
6444,6445,East Garden CDP,NY,9,36,22065,979,"23,591",485,"24,085","23,106",2360.2,38,7.8
6449,6450,Mifflintown borough,PA,9,42,49304,861,"3,630",396,"4,095","3,234",375.6,207,52.3
6453,6454,Wrightstown borough,NJ,9,34,82960,748,"6,698",349,"7,097","6,349",848.8,75,21.5


In [20]:
commuter_df_NYNJCTPA[['Area Name','St1']].rename(columns={'Area Name':'City', 'St1':'State'}).to_csv(fig_path + 'zipcodes.csv')

In [78]:
commuter_df_NYNJCTPA['City'].apply(lambda x: x.replace('CDP', '').replace('(balanced)', '').strip())

0                     New York
4                 Philadelphia
51                  Pittsburgh
57                     Buffalo
62                      Newark
                 ...          
6439    McConnellsburg borough
6444               East Garden
6449       Mifflintown borough
6453       Wrightstown borough
6474         Teterboro borough
Name: City, Length: 1061, dtype: object

In [13]:
driver = webdriver.Chrome()

In [14]:
soup = get_soup_driver('https://www.zip-codes.com/search.asp?fld-city=new+york&fld-state=ny')

In [28]:
table = soup.find_all('table')
df = pd.read_html(str(table))[1]

In [29]:
display(df)

,Zip Code,Area Code,City,County,State,Country
0,10001,646/718/332/917/929/212/347,NEW YORK,NEW YORK,NY,US
1,10002,718,NEW YORK,NEW YORK,NY,US
2,10003,212/646/347/917/332/718/929,NEW YORK,NEW YORK,NY,US
3,10004,212/917/718/646/347,NEW YORK,NEW YORK,NY,US
4,10005,212/929/646/917/332/718/347,NEW YORK,NEW YORK,NY,US
5,10006,917/929,NEW YORK,NEW YORK,NY,US
6,10007,929/212/646/917/347/332/718,NEW YORK,NEW YORK,NY,US
7,10008,212/929/332/646/917,NEW YORK,NEW YORK,NY,US
8,10009,212/718/332/646/917/929,NEW YORK,NEW YORK,NY,US
9,10010,917/646/212/332/718/929,NEW YORK,NEW YORK,NY,US


In [192]:
driver = webdriver.Chrome()

In [193]:
soup = get_soup_driver('https://www.zip-codes.com/search.asp?fld-city=new+york&fld-state=ny')

In [194]:
link = driver.find_element_by_xpath('/html/body/table/tbody/tr/td[2]/div/div[4]/div[1]/a[1]').click()

Get all zipcodes for cities that have more than 50 zipcodes. You have to search more than 1 page. For the below code, I will also remove duplicates because if a city doesn't have a page 3, it will simply show you page 1.

In [195]:
def get_zips_all(city, state, scraped_zips, page=1):
    city = urllib.parse.quote(city, safe='~')
    if page == 1:
        base_url = f'https://www.zip-codes.com/search.asp?fld-city={city}&fld-state={state}'
    else:
        base_url = f'https://www.zip-codes.com/search.asp?fld-city={city}&fld-state={state}' + f'&pg={page}'
    soup = get_soup_driver(base_url)
    table = soup.find_all('table')
    df = pd.read_html(str(table))[1]
    city = urllib.parse.unquote(city)
    df['Levenshtein'] = df.City.apply(lambda x: lev.ratio(x, city.upper()))
    df['Zip Code'] = df['Zip Code'].apply(lambda x: str(x))
    df['Zip Code'] = df['Zip Code'].apply(lambda x: '0' + str(x) if len(str(x)) == 4 else x)
    
    scraped_zips[city+' '+state] = df.to_dict()
    pickling_on = open(fig_path + "scraped_zips.pickle","wb")
    pickle.dump(scraped_zips, pickling_on)
    pickling_on.close()
#     return df[df.Levenshtein > 0.9]
    return df[['City', 'State', 'Zip Code']][df.Levenshtein > 0.9]

In [198]:
def loop_zips_all(path_city_list, path_zip_list, end, scraped_zips, page): # put in path to zip code df in progress and start index
    city_df = pd.read_csv(path_city_list, index_col=0) # this is df master list of cities to be searched for zip
    zip_df = pd.read_csv(path_zip_list, index_col=0) # this is df containing zips that gets added onto each loop
    
    cities = list(city_df.City)[:end]
#     print(cities)
    states = list(city_df.State)[:end]
    indices = list(city_df.index)[:end]
    
    global driver
    driver = webdriver.Chrome()
    
    counter = 0
    
    for i, city in enumerate(cities):
        try:
            sub_df = get_zips_all(city, states[i], scraped_zips, page)
            zip_df = zip_df.append(sub_df, ignore_index=True)
            city_df.loc[indices[i], 'Done'] = 1
            city_df.to_csv(path_city_list)
            zip_df.to_csv(path_zip_list)
            counter += 1
#             if counter == 2: break
        except IndexError:
            driver.quit()
            return df
        except:
            continue
    driver.quit()
    return df
    #     print(start)
#     display(df) 
        
    

In [199]:
def run_loop_zips_all(cities_path, zips_path, scraped_zips, end_index, counter=0):
    df = pd.read_csv(cities_path, index_col=0)
    df.reset_index(drop=True, inplace=True)
#     start_index = list(df[df.Done.isna()].index)[0]
    try:
        loop_zips_all(cities_path, zips_path, end_index, scraped_zips, 2)
        loop_zips_all(cities_path, zips_path, end_index, scraped_zips, 3)
    except:
        counter += 1
        if counter == 5:
            print(counter)
            raise Exception
        time.sleep(5)
        run_loop_zips_all(cities_path, zips_path, scraped_zips, counter)

In [201]:
run_loop_zips_all(fig_path + 'cities.csv', fig_path + 'zips.csv', scraped_zips, 4)

In [203]:
zips_df = pd.read_csv(fig_path + 'zips.csv', index_col=0)

In [206]:
zips_df.drop_duplicates().to_csv(fig_path + 'zips.csv')

There are zips that weren'et collected in the first round because they do not correspond to city names yet there is EV data on them. We need to collect a full list of zips from unitedstateszipcodes.org and collect all demographics from each one.

In [263]:
def collect_state_zips(state): # enter state abbreviation
    url = f'https://www.unitedstateszipcodes.org/{state.lower()}'
    soup = get_soup_driver(url)
    result = {}
    result['City'] = ['']
    result['State'] = ['']
    result['Zip Code'] = ['']
    cities = soup.findAll(class_='list-group-item')
    for city in cities:
        result['City'] += [city.find(class_='prefix-col3').text.replace('\n', '').replace('\t', '').strip()]
        result['State'] += [state.upper()]
        result['Zip Code'] += [city.find(class_='prefix-col1').text.replace('\n', '').replace('\t', '').strip()]
    result = pd.DataFrame(result)
    return result.iloc[1:]

In [264]:
driver = webdriver.Chrome()
ny_zips = collect_state_zips('ny')
nj_zips = collect_state_zips('nj')
ct_zips = collect_state_zips('ct')
md_zips = collect_state_zips('md')
driver.quit()

In [207]:
driver = webdriver.Chrome()
base_url = 'https://www.unitedstateszipcodes.org/ny/'
soup = get_soup_driver(base_url)

In [255]:
state = 'ny'
result = {}
result['City'] = ['']
result['State'] = ['']
result['Zip Code'] = ['']
cities = soup.findAll(class_='list-group-item')
for city in cities:
    result['City'] += [city.find(class_='prefix-col3').text.replace('\n', '').replace('\t', '').strip()]
    result['State'] += [state.upper()]
    result['Zip Code'] += [city.find(class_='prefix-col1').text.replace('\n', '').replace('\t', '').strip()]

In [277]:
zips_NYNJCTMD = pd.concat([ct_zips, nj_zips, md_zips, ny_zips])
zips_NYNJCTMD

,City,State,Zip Code
1,Avon,CT,06001
2,Bloomfield,CT,06002
3,Windsor,CT,06006
4,Bristol,CT,06010
5,Bristol,CT,06011
...,...,...,...
2205,Elmira,NY,14902
2206,"Elmira, Elmira Heights, Elmira Hgts, Elmira Hts",NY,14903
2207,Elmira,NY,14904
2208,Elmira,NY,14905


Merge with old zips list

In [286]:
def clean_zips(zipp):
    zipp = str(int(zipp))
#     zipp = str(zipp)[:-2]
    if len(zipp) < 5:
        zipp = '0' + zipp
    return zipp

In [299]:
zips = zips.drop_duplicates(subset='Zip Code')
zips.to_csv('Figures/zips.csv')